# Thin Plate Splines Transformations
**Author: Matthew Beck** \
\
**Purpose:** The purpose of this experiment to implement thin plate spline transformations on images. The goal is to help normalize images of charts that have been folded or bent. TPS registration will be performed post-homography

In [1]:
# imports
import cv2
import numpy as np

In [ ]:
def tps_tranform(image, src_points: List[BoundingBox], dst_points: List[BoundingBox]):

    # convert list of bounding boxes to lists of cv2.KeyPoints
    # we will use the top left of the bounding box as our point of reference
    src_keypoints = [cv2.KeyPoint(bb.left, bb.top, 1) for bb in src_points]
    dst_keypoints = [cv2.KeyPoint(bb.left, bb.top, 1) for bb in dst_points]

    # Thin Plate Splines Transformer from cv2
    tps = cv2.createThinPlateSplineShapeTransformer()

    # get matches between the source and destination points
    # !! this is assuming that the corresponding source and destination points are the same indice in their respective list
    # TODO: filter by category and filter out for only unique
    # TODO: make the matches based off of these unique categories
    # TODO: Dict category (key) : BoundingBox(value)
    matches = [cv2.DMatch(i, i, 0) for i in range(len(src_points))]

    # estimation of the transformation
    tps.estimateTransformation(dst_keypoints, src_keypoints, matches)

    # Alter the image according to the transformation
    h, w, _ = image.shape
    # create grid
    x = np.linspace(0, w-1, w)
    y = np.linspace(0, h-1, h)
    grid_x, grid_y = np.meshgrid(x, y)
    grid = np.concatenate([grid_x.flatten().reshape(-1,1), grid_y.flatten().reshape(-1, 1)], axis=1)

    # apply the transformation
    tranformed_points = tps.applyTransformation(grid.astype(np.float32))

    # reshape into grid
    transformed_x = np.float32(tranformed_points[:, 0].reshape(grid_y.shape))
    transformed_y = np.float32(tranformed_points[:, 1].reshape(grid_x.shape))

    # warp the image
    warp_img = cv2.remap(image, transformed_x, transformed_y, interpolation=cv2.INTER_LINEAR)

    return warp_img